In [1]:
%%time

import pandas as pd
import numpy as np
from scipy import sparse
import tarfile as tar
import shutil
import os

CPU times: user 1.49 s, sys: 2.81 s, total: 4.3 s
Wall time: 355 ms


In [2]:
%%time

with tar.open("Webscope_C15.tgz", "r") as tar_ref:
    tar_ref.extractall()

CPU times: user 7.36 s, sys: 1.58 s, total: 8.94 s
Wall time: 15.8 s


In [3]:
%%time

with tar.open("ydata-ymusic-kddcup-2011-track1.tar.bz2", "r") as tar_ref:
    tar_ref.extractall()

CPU times: user 3min 32s, sys: 5.64 s, total: 3min 38s
Wall time: 3min 39s


In [4]:
%%time

fileIN = open("ydata-ymusic-kddcup-2011-track1/validationIdx1.txt", "r+")
fileOUT = open("test_tmp.txt", "w+")


lineCount = 0
for line in fileIN:
    if lineCount == 0:
        token = line.strip().split('|')
        uid = token[0]
        lineCount = int(token[1])
    else:
        token = line.strip().split()
        iid = token[0]
        score = token[1]
        lineCount = lineCount - 1;
        fileOUT.write(str(uid) + "\t" + str(iid) + "\t" + str(score) + "\n")

fileIN.close()
fileOUT.close()

CPU times: user 8.24 s, sys: 140 ms, total: 8.38 s
Wall time: 8.38 s


In [5]:
%%time

fileIN = open("ydata-ymusic-kddcup-2011-track1/trainIdx1.txt", "r+")
fileOUT = open("train_tmp.txt", "w+")

lineCount = 0
for line in fileIN:
    if lineCount == 0:
        token = line.strip().split('|')
        uid = token[0]
        lineCount = int(token[1])
    else:
        token = line.strip().split()
        iid = token[0]
        score = token[1]
        lineCount = lineCount - 1;
        fileOUT.write(str(uid) + "\t" + str(iid) + "\t" + str(score) + "\n")

fileIN.close()
fileOUT.close()

CPU times: user 8min, sys: 6.77 s, total: 8min 7s
Wall time: 8min 8s


In [6]:
%%time

shutil.rmtree("ydata-ymusic-kddcup-2011-track1/")
os.remove("ydata-ymusic-kddcup-2011-track1.tar.bz2")
os.remove("ydata-ymusic-kddcup-2011-track2.tar.bz2")
os.remove("README")
os.remove("WebscopeReadMe.txt")

CPU times: user 0 ns, sys: 809 ms, total: 809 ms
Wall time: 811 ms


In [7]:
m = 1_000_990
n = 624_961

nnz_train = 252_800_275
nnz_test = 4_003_960

train_data_file = "train_tmp.txt"
test_data_file = "test_tmp.txt"

print("preparing test data")
%time test_data = pd.read_csv(test_data_file, delimiter = "\t",header=None)
test_user = test_data[0].values
test_item = test_data[1].values
test_rating = test_data[2].values

print("preparing training data")
%time train_data = pd.read_csv(train_data_file, delimiter = "\t",header=None)
train_user = train_data[0].values
train_item = train_data[1].values
train_rating = train_data[2].values

preparing test data
CPU times: user 755 ms, sys: 172 ms, total: 927 ms
Wall time: 929 ms
preparing training data
CPU times: user 47.9 s, sys: 9.71 s, total: 57.6 s
Wall time: 57.6 s


In [8]:
%%time

os.remove("test_tmp.txt")
os.remove("train_tmp.txt")

CPU times: user 0 ns, sys: 480 ms, total: 480 ms
Wall time: 481 ms


In [9]:
%%time

print(test_user)
print(test_item)
print(test_rating)
print("")
print(np.max(test_user))
print(np.max(test_item))
print(np.max(test_rating))
print("")
print(np.min(test_user))
print(np.min(test_item))
print(np.min(test_rating))
print("")
print(np.unique(test_user).size)
print(np.unique(test_item).size)
print(np.unique(test_rating).size)
print("")
print(test_user.size)

assert test_user.size == nnz_test

[      0       0       0 ... 1000989 1000989 1000989]
[540429 554898 470288 ... 569179 574204 578821]
[70 90 70 ... 90 90 90]

1000989
624960
100

0
2
0

1000990
258694
101

4003960
CPU times: user 401 ms, sys: 23.8 ms, total: 425 ms
Wall time: 422 ms


In [10]:
%%time

print(train_user)
print(train_item)
print(train_rating)
print("")
print(np.max(train_user))
print(np.max(train_item))
print(np.max(train_rating))
print("")
print(np.min(train_user))
print(np.min(train_item))
print(np.min(train_rating))
print("")
print(np.unique(train_user).size)
print(np.unique(train_item).size)
print(np.unique(train_rating).size)
print("")
print(train_user.size)

assert train_user.size == nnz_train
assert np.max(train_user)+1 == m  # ids start with 0
assert np.max(train_item)+1 == n  # ids start with 0

[      0       0       0 ... 1000989 1000989 1000989]
[507696 137915  22757 ... 531386 532569 556366]
[90 90 90 ... 80 90 90]

1000989
624960
100

0
0
0

1000990
624961
101

252800275
CPU times: user 30.8 s, sys: 3.27 s, total: 34.1 s
Wall time: 34.1 s


In [11]:
%%time

R_test_coo = sparse.coo_matrix((test_rating, (test_user, test_item)))
assert R_test_coo.nnz == nnz_test

# 0-based to 1-based
outfile_test = open("test.txt", 'w')
for i in range(nnz_test):
    outfile_test.write(str(test_user[i] + 1) + " " + str(test_item[i] + 1) + " " + str(test_rating[i]) + "\n")
outfile_test.close()

CPU times: user 13 s, sys: 121 ms, total: 13.2 s
Wall time: 13.1 s


In [12]:
%%time
# for test data, we need COO format to calculate test RMSE

R_test_coo.data.astype(np.float32).tofile('R_test_coo.data.bin')
R_test_coo.row.tofile('R_test_coo.row.bin')
R_test_coo.col.tofile('R_test_coo.col.bin')

test_data = np.fromfile('R_test_coo.data.bin', dtype=np.float32)
test_row = np.fromfile('R_test_coo.row.bin', dtype=np.int32)
test_col = np.fromfile('R_test_coo.col.bin', dtype=np.int32)

CPU times: user 8.78 ms, sys: 63.9 ms, total: 72.7 ms
Wall time: 162 ms


In [13]:
%%time

print(R_test_coo.data)
print(R_test_coo.row)
print(R_test_coo.col)
print("")
print(test_data)
print(test_row)
print(test_col)

[70 90 70 ... 90 90 90]
[      0       0       0 ... 1000989 1000989 1000989]
[540429 554898 470288 ... 569179 574204 578821]

[70. 90. 70. ... 90. 90. 90.]
[      0       0       0 ... 1000989 1000989 1000989]
[540429 554898 470288 ... 569179 574204 578821]
CPU times: user 3.36 ms, sys: 0 ns, total: 3.36 ms
Wall time: 2.75 ms


In [14]:
%%time

print(np.max(R_test_coo.data))
print(np.max(R_test_coo.row))
print(np.max(R_test_coo.col))
print("")
print(np.min(R_test_coo.data))
print(np.min(R_test_coo.row))
print(np.min(R_test_coo.col))
print("")
print(np.unique(test_user).size)
print(np.unique(R_test_coo.row).size)
print(np.unique(test_item).size)
print(np.unique(R_test_coo.col).size)

100
1000989
624960

0
0
2

1000990
1000990
258694
258694
CPU times: user 614 ms, sys: 11.6 ms, total: 626 ms
Wall time: 620 ms


In [15]:
%%time

R_train_coo = sparse.coo_matrix((train_rating, (train_user, train_item)))
assert R_train_coo.nnz == nnz_train

# 0-based to 1-based
outfile_train = open("train.txt", 'w')
for i in range(nnz_train):
    outfile_train.write(str(train_user[i] + 1) + " " + str(train_item[i] + 1) + " " + str(train_rating[i]) + "\n")
outfile_train.close()

CPU times: user 14min 24s, sys: 11.2 s, total: 14min 35s
Wall time: 14min 31s


In [16]:
%%time

# for training data, we need COO format to calculate training RMSE
# we need CSR format R when calculate X from \Theta
# we need CSC format of R when calculating \Theta from X
R_train_coo.data.astype(np.float32).tofile('R_train_coo.data.bin')
R_train_coo.row.tofile('R_train_coo.row.bin')
R_train_coo.col.tofile('R_train_coo.col.bin')

R_train_csr = R_train_coo.tocsr()
R_train_csc = R_train_coo.tocsc()

R_train_csr.data.astype(np.float32).tofile('R_train_csr.data.bin')
R_train_csr.indices.tofile('R_train_csr.indices.bin')
R_train_csr.indptr.tofile('R_train_csr.indptr.bin')
R_train_csc.data.astype(np.float32).tofile('R_train_csc.data.bin')
R_train_csc.indices.tofile('R_train_csc.indices.bin')
R_train_csc.indptr.tofile('R_train_csc.indptr.bin')

CPU times: user 45.4 s, sys: 11.4 s, total: 56.7 s
Wall time: 1min


In [17]:
%%time

train_data = np.fromfile('R_train_coo.data.bin', dtype=np.float32)
train_row = np.fromfile('R_train_coo.row.bin', dtype=np.int32)
train_col = np.fromfile('R_train_coo.col.bin', dtype=np.int32)

train_csc_data = np.fromfile('R_train_csc.data.bin', dtype=np.float32)
train_csc_indices = np.fromfile('R_train_csc.indices.bin', dtype=np.int32)
train_csc_indptr = np.fromfile('R_train_csc.indptr.bin', dtype=np.int32)

train_csr_data = np.fromfile('R_train_csr.data.bin', dtype=np.float32)
train_csr_indices = np.fromfile('R_train_csr.indices.bin', dtype=np.int32)
train_csr_indptr = np.fromfile('R_train_csr.indptr.bin', dtype=np.int32)

CPU times: user 38.5 ms, sys: 5.42 s, total: 5.46 s
Wall time: 41.5 s


In [18]:
%%time

print(R_train_coo.data)
print(R_train_coo.row)
print(R_train_coo.col)
print("")
print(train_data)
print(train_row)
print(train_col)
print("")
print(R_train_csr.data)
print(R_train_csr.indices)
print(R_train_csr.indptr)
print("")
print(train_csr_data)
print(train_csr_indices)
print(train_csr_indptr)
print("")
print(R_train_csc.data)
print(R_train_csc.indices)
print(R_train_csc.indptr)
print("")
print(train_csc_data)
print(train_csc_indices)
print(train_csc_indptr)

[90 90 90 ... 80 90 90]
[      0       0       0 ... 1000989 1000989 1000989]
[507696 137915  22757 ... 531386 532569 556366]

[90. 90. 90. ... 80. 90. 90.]
[      0       0       0 ... 1000989 1000989 1000989]
[507696 137915  22757 ... 531386 532569 556366]

[90 90 70 ... 80 90 90]
[  9101  22757  26137 ... 531386 532569 556366]
[        0        41       270 ... 252800219 252800246 252800275]

[90. 90. 70. ... 80. 90. 90.]
[  9101  22757  26137 ... 531386 532569 556366]
[        0        41       270 ... 252800219 252800246 252800275]

[50 83 71 ... 90  0 90]
[  2435 101039 208370 ... 951381 951516 987296]
[        0        20        49 ... 252800191 252800246 252800275]

[50. 83. 71. ... 90.  0. 90.]
[  2435 101039 208370 ... 951381 951516 987296]
[        0        20        49 ... 252800191 252800246 252800275]
CPU times: user 0 ns, sys: 13.3 ms, total: 13.3 ms
Wall time: 185 ms


In [19]:
%%time

print(np.max(R_train_coo.data))
print(np.max(R_train_coo.row))
print(np.max(R_train_coo.col))
print("")
print(np.min(R_train_coo.data))
print(np.min(R_train_coo.row))
print(np.min(R_train_coo.col))
print("")
print(np.unique(train_user).size)
print(np.unique(R_train_coo.row).size)
print(np.unique(train_item).size)
print(np.unique(R_train_coo.col).size)

100
1000989
624960

0
0
0

1000990
1000990
624961
624961
CPU times: user 45.7 s, sys: 7.07 s, total: 52.8 s
Wall time: 52.8 s


In [20]:
%%time
#NNZ by cols
print(len(R_train_csr.getnnz(axis=0)))
print(R_train_csr.getnnz(axis=0))
print("")
print(np.min(R_train_csr.getnnz(axis=0)))
print(np.max(R_train_csr.getnnz(axis=0)))
print("")
print(np.mean(R_train_csr.getnnz(axis=0)))

624961
[ 20  29  81 ... 288  55  29]

4
463820

404.50568115450403
CPU times: user 8.82 s, sys: 4.75 s, total: 13.6 s
Wall time: 13.6 s


In [21]:
%%time
#NNZ by rows
print(len(R_train_csr.getnnz(axis=1)))
print(R_train_csr.getnnz(axis=1))
print("")
print(np.min(R_train_csr.getnnz(axis=1)))
print(np.max(R_train_csr.getnnz(axis=1)))
print("")
print(np.mean(R_train_csr.getnnz(axis=1)))

1000990
[ 41 229 101 ...  21  27  29]

10
307201

252.55025025225027
CPU times: user 3.06 ms, sys: 7.97 ms, total: 11 ms
Wall time: 8.31 ms


In [22]:
%%time

print("write extra meta file")

outfile_meta = open("meta_modified_all", 'w')
outfile_meta.write(str(m) + " " + str(n) + "\n" + str(nnz_train) + "\n")
outfile_meta.write("""R_train_coo.data.bin
R_train_coo.row.bin
R_train_coo.col.bin
R_train_csr.indptr.bin
R_train_csr.indices.bin
R_train_csr.data.bin
R_train_csc.indptr.bin
R_train_csc.indices.bin
R_train_csc.data.bin
""")
outfile_meta.write(str(nnz_test) + "\n")
outfile_meta.write("""R_test_coo.data.bin
R_test_coo.row.bin
R_test_coo.col.bin
""")
outfile_meta.close()

write extra meta file
CPU times: user 1.35 ms, sys: 0 ns, total: 1.35 ms
Wall time: 745 µs


In [23]:
%%time

print("writing extra meta file")

outfile_meta = open("meta", 'w')
outfile_meta.write(str(m) + " " + str(n) + "\n")
outfile_meta.write(str(nnz_train) + " " + "train.txt\n")
outfile_meta.write(str(nnz_test) + " " + "test.txt\n")
outfile_meta.close()

writing extra meta file
CPU times: user 794 µs, sys: 32 µs, total: 826 µs
Wall time: 578 µs
